In [2]:
import pysal as ps
import pandas as pd
import numpy as np
import os
os.chdir("/media/sf_Shared_Linux/recurrence_interval/")
from dataIO import dbf2df, df2dbf

In [3]:
os.chdir("/media/sf_Shared_Linux/Project/Data/")
df = dbf2df("S1_100yr.dbf")

In [5]:
df.T

0     \
Active_ind                                  A   
AnType                                    HAZ   
AnalysisOp                                  0   
BM_OBJID                                56023   
BM_OBJID_1                              56023   
BldgDmgPct                            5.90717   
BldgLoss                              9927.03   
CensusBloc                    518100418024039   
ContDmgPct                             7.4886   
ContLoss                              6292.32   
ControlHaz                                  R   
DDF_Zone                                  RVR   
InvLoss                                     0   
OBJECTID                                    1   
OBJECTID_1                              68294   
ReturnPeri                                100   
SHAPE_Area                            1406.06   
SHAPE_Leng                            150.215   
SOccupId                                 R12N   
Stories                                     2   
StudyCaseI                                  9   
Tract                             51810041802   
UDF_Addres                  2269 GREENWELL RD   
UDF_Area                                 1440   
UDF_Backup                                  0   
UDF_BldgDa                                107   
UDF_BldgTy                               Wood   
UDF_City                       Virginia Beach   
UDF_Commen                              56023   
UDF_ContDa                                 23   
UDF_Contac  BYERLEY RONALD T & BYERLEY FARM S   
UDF_Conten                            84025.3   
UDF_Cost                               168051   
UDF_Design                                  3   
UDF_FirstF                                  1   
UDF_FloodP                                  0   
UDF_Founda                                  5   
UDF_InvDam                                  0   
UDF_Latitu                             36.908   
UDF_Longit                            -76.129   
UDF_Name    BYERLEY RONALD T & BYERLEY FARM S   
UDF_NumSto                                  1   
UDF_Occupa                               RES1   
UDF_PhoneN                                      
UDF_Shelte                                  0   
UDF_State                                  VA   
UDF_YearBu                               1983   
UDF_ZipCod                              23455   
UserDefi_1                              56023   
UserDefine                              56023   

                                           1                             2     \
Active_ind                                    A                             A   
AnType                                      HAZ                           HAZ   
AnalysisOp                                    0                             0   
BM_OBJID                                  55972                         57814   
BM_OBJID_1                                55972                         57814   
BldgDmgPct                              8.63831                             0   
BldgLoss                                24699.2                             0   
CensusBloc                      518100418024039               518100418024031   
ContDmgPct                              6.91064                             0   
ContLoss                                9879.68                             0   
ControlHaz                                    R                             R   
DDF_Zone                                    RVR                           RVR   
InvLoss                                       0                             0   
OBJECTID                                      2                             3   
OBJECTID_1                                11797                         59744   
ReturnPeri                                  100                           100   
SHAPE_Area                              2113.64                       590.093   
SHAPE_Leng                              209.075                       103.566   
SO

In [2]:
def SummarizeHAZUS(InputDBF,OutputDBF):
    df = dbf2df(InputDBF)
    df["CensusBloc"]=df["CensusBloc"].astype(str)
    df.set_index("CensusBloc",inplace=True)

    bldgbyblock = {}
    contbyblock = {}
    countbyblock= {}

    for i in range(len(df.index.unique())):
        censusblock = df.index.unique()[i]
        bldgloss = df.ix[censusblock].BldgLoss.sum()
        contloss = df.ix[censusblock].ContLoss.sum()
        bldcount = len(df.BldgLoss[df.index == censusblock])
        bldgbyblock[censusblock] = bldgloss
        contbyblock[censusblock] = contloss
        countbyblock[censusblock]= bldcount

    df_cont = pd.Series(contbyblock, name='ContLoss')
    df_cont.index.name = 'CensusBlock'
    df_bldg = pd.Series(bldgbyblock, name='BldgLoss')
    df_bldg.index.name = 'CensusBlock'
    df_count = pd.Series(countbyblock, name='BldCount')
    df_count.index.name = 'CensusBlock'

    losses = pd.concat([df_bldg,df_cont,df_count],axis=1)
    losses.reset_index(inplace=True)
    losses['CensusBlock'] = losses['CensusBlock'].astype(str)

    my_specs = [('C', 17, 17), ('N', 9, 3), ('N', 9, 3),('N', 3, 0)]
    df2dbf(losses,OutputDBF,my_specs=my_specs)

In [3]:
indbf = ["Data/S0_100yr.dbf","Data/S0_10yr.dbf","Data/S1_10yr.dbf","Data/S1_100yr.dbf"]
outdbf =["Data/S0_100yr_sum.dbf","Data/S0_10yr_sum.dbf","Data/S1_10yr_sum.dbf","Data/S1_100yr_sum.dbf"]
for i in range(4):
    SummarizeHAZUS(indbf[i],outdbf[i])